In [21]:
import pathlib as pl
import sys
import json
import subprocess as sp

import pandas as pd
import numpy as np
import payulator as pl
import voluptuous as vt

ROOT = Path("../")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
YEAR = 2020
YEAR_LABEL = f"{YEAR}0701--{YEAR + 1}0630"
AML_DIR = pl.Path(f"../aml/reports/{YEAR_LABEL}")
assert AML_DIR.exists()

In [8]:
def clean_transactions(raw_transactions):
    f = raw_transactions.copy()
    return (
        f
        .rename(columns={
            c: c.lower().replace(" ", "_").replace("(", "").replace(")", "")
            for c in f.columns
        })
        .rename(columns={"memo/description": "description"})
    )

def remove_seed_funding(transactions):
    return transactions.loc[
        lambda x: ~x.description.str.contains("TRANSFER FROM A RAICHEV", case=False)
    ].copy()

def remove_internal_transfers(transactions):
    return transactions.loc[
        lambda x: ~x.description.str.contains("TRANSFER FROM MERRIWEATHER ", case=False)
    ].copy()

def summarize(transactions):
    f = transactions.copy()
    d = {}
    d["num_credits"] = f.amount_credit.dropna().count()
    d["num_debits"] = f.amount_debit.dropna().count()
    d["num_transactions"] = d["num_credits"] + d["num_debits"]  
    d["num_credits_per_month"] = d["num_credits"]/12  # Use date next time
    d["num_debits_per_month"] = d["num_debits"]/12  # Use date next time
    d["value_credits"] = f.amount_credit.sum()
    d["value_debits"] = f.amount_debit.sum()
    d["value_transactions"] = d["value_credits"] + d["value_debits"] 
    d["value_credits_per_month"] = d["value_credits"]/12  # Use date next time
    d["value_debits_per_month"] = d["value_debits"]/12  # Use date next time
    d["num_crediting_accounts"] = f.op_bank_account_number.nunique()
    return pd.Series(d)

In [20]:
path = AML_DIR/f"transactions_{YEAR_LABEL}.csv"
transactions = clean_transactions(pd.read_csv(path))
display(transactions.head(10))

display(transactions.pipe(summarize))

display(transactions.pipe(remove_seed_funding).pipe(remove_internal_transfers).pipe(summarize))


,account_number,date,description,source_code_payment_type,tp_ref,tp_part,tp_code,op_ref,op_part,op_code,op_name,op_bank_account_number,amount_credit,amount_debit,amount,balance
0,38-9019-0508016-00,03-07-2020,TRADEME 41F3 PING Wellington ;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181.00,-181.00,37974.04
1,38-9019-0508016-00,06-07-2020,Automatic Payment ebike shop maurice loan Elec...,AP,ebike shop,maurice loan,NaN,NaN,NaN,NaN,Electric Bicycl,03-0104-0831837-00,250.00,NaN,250.00,38224.04
2,38-9019-0508016-00,06-07-2020,Automatic Payment 000020190604Loan Catherine V...,AP,000020190604,Loan,Catherine,NaN,NaN,NaN,VALLYON I&C,06-0317-0073961-00,634.06,NaN,634.06,38858.10
3,38-9019-0508016-00,06-07-2020,Automatic Payment I HARTMANN VOLVO 2019 INGRID...,AP,I HARTMANN,VOLVO 2019,NaN,NaN,NaN,NaN,INGRID HARTMANN,11-5392-0320144-00,354.42,NaN,354.42,39212.52
4,38-9019-0508016-00,06-07-2020,AP#16111598 FROM RESILIO LIMITED ;Payment from...,NaN,20190514,Resilio,NaN,NaN,NaN,NaN,RESILIO LIMITED,38-9016-0667089-00,1569.50,NaN,1569.50,40782.02
5,38-9019-0508016-00,06-07-2020,IFSO WLG ;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414.00,-414.00,40368.02
6,38-9019-0508016-00,07-07-2020,Rebel Sports Mt Albert ;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181.98,-181.98,40186.04
7,38-9019-0508016-00,08-07-2020,TRANSFER FROM A RAICHEV - 00 ;rain pants reimb...,NaN,NaN,rain pants,reimburse,NaN,NaN,NaN,A RAICHEV,38-9009-0530201-00,91.00,NaN,91.00,40277.04
8,38-9019-0508016-00,15-07-2020,2degrees Mobile Auckland ;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.00,-10.00,40267.04
9,38-9019-0508016-00,31-07-2020,TRANSACTION FEES ;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.00,-5.00,40262.04


num_credits                    80.000000
num_debits                     72.000000
num_transactions              152.000000
num_credits_per_month           6.666667
num_debits_per_month            6.000000
value_credits              103318.610000
value_debits               137273.720000
value_transactions         240592.330000
value_credits_per_month      8609.884167
value_debits_per_month      11439.476667
num_crediting_accounts         14.000000
dtype: float64

num_credits                    75.000000
num_debits                     72.000000
num_transactions              147.000000
num_credits_per_month           6.250000
num_debits_per_month            6.000000
value_credits               41440.750000
value_debits               137273.720000
value_transactions         178714.470000
value_credits_per_month      3453.395833
value_debits_per_month      11439.476667
num_crediting_accounts         14.000000
dtype: float64

In [15]:
# Estimate car loan vs other loan revenue
f = transactions.pipe(remove_seed_funding).pipe(remove_internal_transfers)

op_codes_car = [
    "12-3165-0361190-00",
    "03-1500-0132143-00", 
    "11-5392-0320144-00",
]
d = {}
d["revenue"] = f.amount_credit.sum()
d["revenue_car"] = f.loc[lambda x: x.op_bank_account_number.isin(op_codes_car), "amount_credit"].sum()
d["revenue_car_pc"] = 100*d["revenue_car"]/d["revenue"]
d["revenue_other"] = d["revenue"] - d["revenue_car"]
d["revenue_other_pc"] = 100*d["revenue_other"]/d["revenue"]

display(d)

{'revenue': 41440.75,
 'revenue_car': 5290.170000000001,
 'revenue_car_pc': 12.765623209039415,
 'revenue_other': 36150.58,
 'revenue_other_pc': 87.23437679096058}

In [49]:
LOAN_DIRS = [pl.Path("../loans/active"), pl.Path("../loans/finished")]

def collect_active_loans(from_date, to_date, loan_dirs=LOAN_DIRS):
    """
    Given a list of directory paths (list of strings or Path objects),
    search every subdirectory of every given directory for a file
    named ``"payment_schedule.csv"``.
    Read each such payment schedule as a DataFrame and return
    a dictionary of the form loan_directory_name -> payment schedule
    DataFrame.
    """
    active = []
    for root in loan_dirs:
        for p in root.iterdir():
            if not p.is_dir():
                continue
            for q in p.glob("parameters.json"):
                try:
                    loan = pl.build_loan(q)
                except vt.MultipleInvalid as e:
                    print(f"Skipping {p} because of an error in its JSON paremeters.")
                    print(e)
                    continue
                
                with q.open() as src:
                    parameters = json.load(src)
                
                # Calculate number of days overlap between given time window
                # and time window of loan
                schedule = loan.summarize()["payment_schedule"]
                r1 = [pd.to_datetime(from_date), pd.to_datetime(to_date)]
                r2 = [schedule.payment_date.min(), schedule.payment_date.max()]
                latest_start = max(r1[0], r2[0])
                earliest_end = min(r1[1], r2[1])
                delta = (earliest_end - latest_start).days + 1
                overlap = max(0, delta)
                
                # Log as active if positive overlap
                if overlap:
                    active.append({
                        "loan_code": loan.code,
                        "borrowers": parameters["borrowers"],
                        "num_days_active": overlap,
                    })
            
    return active



In [51]:
active_loans = collect_active_loans("2020-07-01", "2021-06-30")
display(active_loans)
unique_borrowers = set(b for loan in active_loans for b in loan["borrowers"])
display(unique_borrowers)

Skipping ../loans/finished/Swinson-20170831 because of an error in its JSON paremeters.
Kind must be one on ['amortized', 'interest_only'] for dictionary value @ data['kind']


[{'loan_code': 'EBT-20210511',
  'borrowers': ['Electric Bike Team Limited (NZCN 6043355)'],
  'num_days_active': 25},
 {'loan_code': 'EBT-20210310',
  'borrowers': ['Electric Bike Team Limited (NZCN 6043355)'],
  'num_days_active': 86},
 {'loan_code': 'C-Vallyon-20190604',
  'borrowers': ['Catherine Vallyon of Waitetuna, New Zealand'],
  'num_days_active': 365},
 {'loan_code': 'Haldane-Willis-20200618',
  'borrowers': ['Jack Haldane-Willis, 32 Constable Rd, Auckland, 0881'],
  'num_days_active': 329},
 {'loan_code': 'Resilio-20190514',
  'borrowers': ['Resilio Limited (NZCN 5373541)'],
  'num_days_active': 365},
 {'loan_code': 'Haldane-Willis-20180203',
  'borrowers': ['Jack Haldane-Willis, 32 Constable Rd, Auckland, 0881',
   'Ursula Grace Williams, 32 Constable Rd, Auckland, 0881'],
  'num_days_active': 221},
 {'loan_code': 'EBT-20200224',
  'borrowers': ['Electric Bike Team Limited (NZCN 6043355)'],
  'num_days_active': 249},
 {'loan_code': 'Resilio-20180123',
  'borrowers': ['Resi

{'Catherine Vallyon of Waitetuna, New Zealand',
 'Electric Bike Team Limited (NZCN 6043355)',
 'Jack Haldane-Willis, 32 Constable Rd, Auckland, 0881',
 'Resilio Limited (NZCN 5373541)',
 'Ursula Grace Williams, 32 Constable Rd, Auckland, 0881'}